In [1]:
# Modules
import pandas as pd
import numpy as np

In [2]:
#set path for file
file="Resources/purchase_data.csv"

In [3]:
#Read data file with pandas library
file_df=pd.read_csv(file)
file_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
#Find the total number of rows in the dataframe
index=file_df.index
totalrows=len(index)
print(totalrows)

780


In [5]:
#find the total number of players
total_players=len(file_df["SN"].value_counts())
total_players

576

In [6]:
#Create a date frame for the total number of players
number_of_players=pd.DataFrame({"Total Players":[total_players]})
number_of_players

,Total Players
0,576


In [7]:
#find the number of unique items, average price, number of purchases, and total revenue
total_uitems=len(file_df["Item ID"].unique())
total_uitems
avg_price=round(file_df["Price"].mean(),2)
avg_price
total_purchases=(file_df["Purchase ID"].count())
total_purchases
total_revenue=(file_df["Price"].sum())

In [8]:
#put above data in a dataframe
summary_df=pd.DataFrame({"Number of Unique Items": [total_uitems], "Average Price": [avg_price], "Number of Purchases":[total_purchases], "Total Revenue":[total_revenue]})

In [9]:
#format the summary table using currency
summary_df.style.format({'Average Price':"${:,.2f}",
                         'Total Revenue': "${:,.2f}"})
summary_df.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.05,780,2379.77


In [10]:
#Gender Demographics

#Total count without repeating SN's
totalcount = file_df["SN"].nunique()

#filter by male, female and other
males = file_df[file_df["Gender"] == "Male"]["SN"].nunique()
females = file_df[file_df["Gender"] == "Female"]["SN"].nunique()
other = totalcount - males - females

#find percentages using calculation
maleperc = ((males/totalcount)*100)
femaleperc = ((females/totalcount)*100)
otherperc = ((other/totalcount)*100)

#Create the dataframe
gender_demo_df = pd.DataFrame({"Gender": ["Male", "Female", "Other / Non-Disclosed"], "Percentage of Players": [maleperc, femaleperc, otherperc],
                                        "Total Count": [males, females, other]}, columns = 
                                        ["Gender", "Percentage of Players", "Total Count"])
#Remove the index numbers in the DF                                        
gender_demo = gender_demo_df.set_index("Gender")

#format the Perecentages
gender_demo.style.format({"Percentage of Players": "{:.2f}%"})

gender_demo.head()


,Percentage of Players,Total Count
Gender,,
Male,84.027778,484
Female,14.062500,81
Other / Non-Disclosed,1.909722,11


In [11]:
#Purchasing Analysis

#Find purchase count, average purchase price, total purchase value, and avg total purchase per person

#total purchases by filtering and .count function
totalFEMpurchase = file_df[file_df["Gender"] == "Female"]["Price"].count()
totalMALEpurchase = file_df[file_df["Gender"] == "Male"]["Price"].count()
totalOTHpurchase = file_df[file_df["Gender"] == "Other / Non-Disclosed"]['Price'].count()

#avg purchase price
avgFEMpurchase = file_df[file_df["Gender"] == "Female"]["Price"].mean()
avgMALEpurchase = file_df[file_df["Gender"] == "Male"]["Price"].mean()
avgOTHpurchase = file_df[file_df["Gender"] == "Other / Non-Disclosed"]['Price'].mean()

#total purchase value
sum_female = file_df[file_df["Gender"] == "Female"]["Price"].sum()
sum_males = file_df[file_df["Gender"] == "Male"]["Price"].sum()
sum_other = file_df[file_df["Gender"] == "Other / Non-Disclosed"]['Price'].sum()

#average total purchase per person
avgtotalFEM = sum_female/females
avgtotalMALE = sum_males/males
avgtotalOTH = sum_other/other

#stick it in a DF
purchase_analysis_df = pd.DataFrame ({"Gender":["Female", "Male", "Other/Non-Disclosed"], "Purchase Count":[totalFEMpurchase, totalMALEpurchase, totalOTHpurchase],
                                     "Average Purchase Price":[avgFEMpurchase, avgMALEpurchase,avgOTHpurchase],
                                     "Total Purchase Value":[sum_female, sum_males, sum_other],
                                     "Average Total Purchase Per Person":[avgtotalFEM, avgtotalMALE, avgtotalOTH]}, 
                                     columns = ["Gender", "Purchase Count", "Average Purchase Price", "Total Purchase Value", "Average Total Purchase Per Person"])

#set index
purchase_analy = purchase_analysis_df.set_index("Gender")

#format the columns
purchase_analy.style.format({"Average Purchase Price": "${:.2f}", "Total Purchase Value": "${:,.2f}", "Average Total Purchase Per Person": "${:.2f}"})

purchase_analy

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other/Non-Disclosed,15,3.346000,50.19,4.562727


In [12]:
#age demographics

#find the min and max age of the data
maxage=file_df["Age"].max()
minage=file_df["Age"].min()

#create bins for age
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 100]

#Create labels for the bins
bin_groups = ["<10", "10-14", "15-19", "20-24","25-29", "30-34", "35-39", "40+"]

#categorize existing players
file_df["Age Summary"]=pd.cut(file_df["Age"], bins, labels=bin_groups)
file_df.head()

#group the data by age groups
grouped_ages=file_df.groupby(["Age Summary"])

#count players only once
grouped_count= grouped_ages["SN"].nunique()

#find percentages
age_percentage = grouped_count/total_players*100
age_percentage

#create a dataframe with the results

age_demographics_df = pd.DataFrame ({"Total Count": grouped_count,
                                    "Percentage of Players": age_percentage})
age_demographics_df['Percentage of Players']= age_demographics_df['Percentage of Players'].astype(float).map("{:.2f}%".format)

age_demographics_df

,Total Count,Percentage of Players
Age Summary,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [13]:
#Purchase Analysis by age 

#bin the purchase_data file by age

#Calculations to find Purchase Count, Avg Purchase Price, Purchase Total per Person
PC = grouped_ages["Purchase ID"].count()
APPA = grouped_ages["Price"].mean()
TPV = grouped_ages["Price"].sum()
ATPPP = TPV/grouped_count

#create a summary df
Purchase_Analysis_Age = pd.DataFrame ({"Purchase Count": PC, 
                                       "Average Purchase Price": APPA, 
                                       "Total Purchase Value": TPV, 
                                       "Avg Total Purchase Per Person":ATPPP})
Purchase_Analysis_Age['Average Purchase Price']= Purchase_Analysis_Age['Average Purchase Price'].astype(float).map("${:.2f}".format)
Purchase_Analysis_Age['Total Purchase Value']= Purchase_Analysis_Age['Total Purchase Value'].astype(float).map("${:,.2f}".format)
Purchase_Analysis_Age['Avg Total Purchase Per Person']= Purchase_Analysis_Age['Avg Total Purchase Per Person'].astype(float).map("${:,.2f}".format)
Purchase_Analysis_Age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Age Summary,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [14]:
#Top Spenders

#group the file by SN 
baller_stats = file_df.groupby("SN")

#total purchases by name
big_baller_count = baller_stats["Purchase ID"].count()

#avg purchase by SN
big_baller_price = baller_stats["Price"].mean()

#total purchase value by SN
big_baller_TPV = baller_stats["Price"].sum()

#create a dataframe and sort the list 
Top_Spenders_df = pd.DataFrame({"Purchase Count": big_baller_count, 
                                "Average Purchase Price": big_baller_price, 
                                "Total Purchase Value": big_baller_TPV})
sorted_Top_Spenders = Top_Spenders_df.sort_values(["Total Purchase Value"], ascending = False).head()

sorted_Top_Spenders['Average Purchase Price']= sorted_Top_Spenders['Average Purchase Price'].astype(float).map("${:.2f}".format)
sorted_Top_Spenders['Total Purchase Value']= sorted_Top_Spenders['Total Purchase Value'].astype(float).map("${:,.2f}".format)
sorted_Top_Spenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [15]:
#Most popular items

popular_items = file_df[["Item ID", "Item Name", "Price"]]
popular_items = popular_items.set_index(["Item ID", "Item Name"])
popular_items = popular_items.groupby(["Item ID", "Item Name"])

popular_items.head()


#calculations

popular_items_count = popular_items["Price"].count()

popular_items_total = popular_items["Price"].sum()

popular_items_avg = popular_items_total / popular_items_count

#dataframe

popular_items_df = pd.DataFrame({"Purchase Count": popular_items_count, 
                                "Item Price": popular_items_avg,
                                "Total Purchase Value": popular_items_total})

sorted_popular_items = popular_items_df.sort_values(["Purchase Count"], ascending = False).head()

sorted_popular_items["Item Price"]= sorted_popular_items["Item Price"].astype(float).map("${:.2f}".format)
sorted_popular_items['Total Purchase Value']= sorted_popular_items['Total Purchase Value'].astype(float).map("${:,.2f}".format)

sorted_popular_items

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [16]:
#Most Profitable Items

sorted_popular_items_TPV = popular_items_df.sort_values (["Total Purchase Value"], ascending = False).head()

sorted_popular_items_TPV

sorted_popular_items_TPV["Item Price"]= sorted_popular_items_TPV["Item Price"].astype(float).map("${:.2f}".format)
sorted_popular_items_TPV['Total Purchase Value']= sorted_popular_items_TPV['Total Purchase Value'].astype(float).map("${:,.2f}".format)

sorted_popular_items_TPV

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
